In [50]:
import os
import zipfile
import json

import pandas as pd
import numpy as np

In [2]:
EXPORT_ZIP_PATH = 'export.zip'
TARGET_DIRECTORY = './data'

with zipfile.ZipFile(EXPORT_ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(TARGET_DIRECTORY)

In [42]:
EXPORT_XML = TARGET_DIRECTORY + '/apple_health_export/export.xml'

if not os.path.isfile(EXPORT_XML):
    raise 'Could not find xml export'

df = pd.read_xml(EXPORT_XML, xpath='//Record[@type="HKQuantityTypeIdentifierHeartRate"]')
df = df[['startDate', 'value']]

In [38]:
export = export.rename(columns={'startDate': 'x', 'value': 'y'})
export['x'] = pd.to_datetime(export['x'])
export = export.set_index('x')
export = export.resample('H').median()
export = export.reset_index()
export['x'] = export['x'].astype(np.int64) // 10 ** 6

38018
2485
               x          y
0  1624435200000   75.00000
1  1624438800000   75.00000
2  1624442400000   79.50000
3  1624446000000   85.88985
4  1624449600000   85.50000
5  1624453200000  115.00000
6  1624456800000   89.00000
7  1624460400000   93.00000
8  1624464000000  113.00000
9  1624467600000   84.00000


/tmp/ipykernel_36643/136341254.py:9: FutureWarning: casting datetime64[ns, pytz.FixedOffset(120)] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  export['x'] = export['x'].astype(np.int64) // 10 ** 6


In [46]:
with open('./data.json', 'w') as export_json:
    export.to_json(path_or_buf=export_json, orient='records')
    
with open('./stats.json', 'w') as export_json:
    json.dump({
        "quantile_95": round(export['y'].quantile(.95)), 
        "quantile_99": round(export['y'].quantile(.01)), 
        "quantile_range": {
            "end": round(export['y'].quantile(.99)), 
            "start": round(export['y'].quantile(.01))
        }, 
        "mean": round(export['y'].mean()), 
        "median": round(export['y'].median())
    }, export_json)